In [1]:
import requests
from bs4 import BeautifulSoup
import os
from tqdm import tqdm
from glob import glob
from datetime import datetime
import dateparser
from unidecode import unidecode

In [2]:
import datetime

date1 = datetime.date(2009, 1, 1)
date2 = datetime.date(2023, 1, 1)

def daterange(date1, date2):
    for n in range(int(date2.year) - int(date1.year)+1):
        yield int(date1.year) + n

start_dt = date1
end_dt = date2
dat = []
for dt in daterange(start_dt, end_dt):
    dat.append(dt)
    
dat

[2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 2020,
 2021,
 2022,
 2023]

In [3]:
all_a = []
for y in tqdm(dat):
    r = requests.get(f'https://eprints.unisza.edu.my/view/year/{y}.html')
    soup = BeautifulSoup(r._content, "lxml")
    p = soup.find_all('p')
    a = [p_.find('a').get('href') for p_ in p]
    all_a.extend(a)

 87%|█████████████████████████████████████▎     | 13/15 [00:17<00:02,  1.32s/it]


AttributeError: 'NoneType' object has no attribute 'get'

In [4]:
all_a = sorted(list(set(all_a)))
len(all_a)

6646

In [7]:
import json

with open('all_a-unisza.json', 'w') as fopen:
    json.dump(all_a, fopen)

In [16]:
def get_pdf(url):
    try:
        r = requests.get(url, timeout = 10.0)
        soup = BeautifulSoup(r._content, "lxml")
        a = soup.find_all('a')
        a = [a_.get('href') for a_ in a]
        a = [a_ for a_ in a if a_ and '.pdf' in a_ and 'haspreview' not in a_]
        a = list(set(a))
        return a
    except Exception as e:
        print(e)
        return None

get_pdf(all_a[])

['http://eprints.unisza.edu.my/1004/1/FH03-FBK-17-08053.pdf']

In [17]:
!rm -rf pages-unisza
!mkdir pages-unisza

In [19]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import os

max_worker = 10

all_a = sorted(all_a)
for i in tqdm(range(0, len(all_a), max_worker)):
    
    filename = os.path.join('pages-unisza', f'{i}.json')
    
    if os.path.exists(filename) and os.path.getsize(filename) > 1000:
        continue
    
    pdfs = []
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(get_pdf, url): url for url in all_a[i: i + max_worker]}

        for future in as_completed(futures):
            r = future.result()
            if r:
                pdfs.extend(r)
    
    with open(filename, 'w') as fopen:
        json.dump(pdfs, fopen)

  2%|█                                         | 16/665 [00:29<19:31,  1.81s/it]

('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


  4%|█▋                                        | 27/665 [00:50<21:01,  1.98s/it]

('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


 15%|██████▏                                   | 98/665 [03:05<18:27,  1.95s/it]

('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


 25%|██████████▎                              | 167/665 [05:09<14:16,  1.72s/it]encoding error : input conversion failed due to input error, bytes 0x9D 0x2C 0x20 0x50
encoding error : input conversion failed due to input error, bytes 0x9D 0x2C 0x20 0x50
100%|█████████████████████████████████████████| 665/665 [14:49<00:00,  1.34s/it]
